In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from subprocess import check_output
import math

df = pd.read_csv('./input/cs448b_ipasn.csv')

# data grouping

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.groupby(['date', 'l_ipn'], as_index=False).sum() # date와 l_ipn 별로 합을 구함
df['date']

In [ ]:
df['yday'] = df['date'].dt.dayofyear
df['wday'] = df['date'].dt.dayofweek

# visualization

In [ ]:
# l_ipn 기준으로 그룹 묶기
ip = []
maxF = []
for i in range(0, 10):
    ip.insert(i, df[df['l_ipn']==i])
    maxF.insert(i, np.max(ip[i]['f']))
ip[0].head()

In [ ]:
count, division = np.histogram(ip[0]['f'], bins=10)
count

In [ ]:
division